### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2022
quarter = 2

today = date.today()
today

datetime.date(2022, 8, 18)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2022 AND quarter <= 2) 
OR (year = 2022-1 AND quarter >= 2+1)
ORDER BY year DESC, quarter DESC


In [3]:
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp.shape

(213, 5)

In [4]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(227, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct.sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
176,THG,2022,Q2,2393114,-24902,2418016,"9,710.13"
46,CPNREIT,2022,Q2,1624073,22899,1601174,"6,992.33"
17,BANPU,2022,Q2,30044483,1875967,28168516,"1,501.55"
201,VNG,2022,Q2,1502115,102807,1399308,"1,361.10"
191,TTA,2022,Q2,5153711,-525001,5678712,"1,081.66"


In [6]:
sql = """
DELETE FROM yr_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year, quarter)
rp = conlt.execute(sql)
rp.rowcount

200

In [7]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
rcds = df_ins.values.tolist()
len(rcds)

208

In [8]:
for rcd in rcds:
    print(rcd)

['ACE', 2022, 'Q2', 1485354, 1391075, 94279, 6.78, 698]
['ADVANC', 2022, 'Q2', 25853429, 27361758, -1508329, -5.51, 6]
['AH', 2022, 'Q2', 1175297, 1109197, 66100, 5.96, 9]
['AIE', 2022, 'Q2', 402015, 495054, -93039, -18.79, 720]
['AIMIRT', 2022, 'Q2', 566114, 425165, 140949, 33.15, 669]
['AIT', 2022, 'Q2', 556639, 542406, 14233, 2.62, 11]
['AJ', 2022, 'Q2', 422747, 591137, -168390, -28.49, 12]
['AMATA', 2022, 'Q2', 2171767, 1098346, 1073421, 97.73, 21]
['ANAN', 2022, 'Q2', -796635, 232533, -1029168, -442.59, 23]
['AOT', 2022, 'Q2', -16784073, -13747749, -3036324, -22.09, 24]
['AP', 2022, 'Q2', 5329148, 4910383, 418765, 8.53, 25]
['ASIAN', 2022, 'Q2', 1048954, 921153, 127801, 13.87, 36]
['ASK', 2022, 'Q2', 1367575, 997509, 370066, 37.1, 38]
['ASP', 2022, 'Q2', 578771, 818663, -239892, -29.3, 40]
['AWC', 2022, 'Q2', 3075541, -1905093, 4980634, 261.44, 699]
['BA', 2022, 'Q2', -8985803, -3401356, -5584447, -164.18, 45]
['BAM', 2022, 'Q2', 2705410, 2043406, 662004, 32.4, 709]
['BANPU', 2022

In [9]:
sql = """INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)

INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [10]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [11]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [12]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
176,THG,2022,Q2,2393114,-24902,2418016,"9,710.13"
46,CPNREIT,2022,Q2,1624073,22899,1601174,"6,992.33"
17,BANPU,2022,Q2,30044483,1875967,28168516,"1,501.55"
201,VNG,2022,Q2,1502115,102807,1399308,"1,361.10"
191,TTA,2022,Q2,5153711,-525001,5678712,"1,081.66"


In [13]:
criteria_2 = df_ins.q_amt_p > 300000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
17,BANPU,2022,Q2,30044483,1875967,28168516,"1,501.55"
104,MAJOR,2022,Q2,2074984,334383,1740601,520.54
127,PSL,2022,Q2,6227064,1206757,5020307,416.02
97,LANNA,2022,Q2,2821353,639500,2181853,341.18
37,CHG,2022,Q2,5610839,1363171,4247668,311.60


In [14]:
criteria_3 = df_ins.Pct > 5.0
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
176,THG,2022,Q2,2393114,-24902,2418016,"9,710.13"
46,CPNREIT,2022,Q2,1624073,22899,1601174,"6,992.33"
17,BANPU,2022,Q2,30044483,1875967,28168516,"1,501.55"
201,VNG,2022,Q2,1502115,102807,1399308,"1,361.10"
191,TTA,2022,Q2,5153711,-525001,5678712,"1,081.66"


In [15]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
17,BANPU,2022,Q2,30044483,1875967,28168516,"1,501.55"
104,MAJOR,2022,Q2,2074984,334383,1740601,520.54
127,PSL,2022,Q2,6227064,1206757,5020307,416.02
97,LANNA,2022,Q2,2821353,639500,2181853,341.18
37,CHG,2022,Q2,5610839,1363171,4247668,311.60
30,BH,2022,Q2,2799073,702109,2096964,298.67
20,BCH,2022,Q2,8548803,2161170,6387633,295.56
154,SKR,2022,Q2,1707617,434395,1273222,293.10
160,SPRC,2022,Q2,14407821,3908828,10498993,268.60
21,BCP,2022,Q2,13208283,3652466,9555817,261.63


In [16]:
df_ins.loc[df_ins_criteria,cols].sort_values(['name'],ascending=[True])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,ACE,2022,Q2,1485354,1391075,94279,6.78
2,AH,2022,Q2,1175297,1109197,66100,5.96
4,AIMIRT,2022,Q2,566114,425165,140949,33.15
7,AMATA,2022,Q2,2171767,1098346,1073421,97.73
10,AP,2022,Q2,5329148,4910383,418765,8.53
11,ASIAN,2022,Q2,1048954,921153,127801,13.87
12,ASK,2022,Q2,1367575,997509,370066,37.10
16,BAM,2022,Q2,2705410,2043406,662004,32.40
17,BANPU,2022,Q2,30044483,1875967,28168516,"1,501.55"
19,BBL,2022,Q2,27306314,19694952,7611362,38.65
